## **1. Tải bộ dữ liệu**

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
!cp /path/to/dataset/on/your/drive.

Mounted at /content/drive
cp: missing destination file operand after '/path/to/dataset/on/your/drive.'
Try 'cp --help' for more information.


In [ ]:
!gdown --id 1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R
To: /content/2cls_spam_text_cls.csv
100% 486k/486k [00:00<00:00, 98.4MB/s]


## **2. Import các thư viện cần thiết**

In [ ]:
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## **3. Đọc dữ liệu**


In [ ]:
DATASET_PATH = '/content/2cls_spam_text_cls.csv'
df = pd.read_csv(DATASET_PATH)
messages = df['Message'].values.tolist()
labels = df['Category'].values.tolist()

## **4. Tiền xử lý dữ liệu**


### **Tiền xử lý dữ liệu đặc trưng:**


In [ ]:
# Chuyển văn bản thành chữ thường
def lowercase(text):
    return text.lower()

# Loại bỏ tất cả dấu câu khỏi văn bản
def punctuation_removal(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Chia văn bản thành các từ
def tokenize(text):
    return nltk.word_tokenize(text)

# Loại bỏ các từ dừng (stopwords) khỏi danh sách từ
def remove_stopwords(tokens):
    stop_words = nltk.corpus.stopwords.words('english')
    return [token for token in tokens if token not in stop_words]

# Thực hiện stemming để giảm từ về dạng gốc
def stemming(tokens):
    stemmer = nltk.PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

# Tiền xử lý văn bản bằng cách áp dụng tất cả các bước trên
def preprocess_text(text):
    text = lowercase(text)
    text = punctuation_removal(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)

    return tokens


In [ ]:
messages = [preprocess_text(message) for message in messages]

In [ ]:
# Tạo từ điển chứa tất cả các từ độc nhất từ danh sách các tin nhắn
def create_dictionary(messages):
    dictionary = []  # Khởi tạo danh sách rỗng để lưu trữ các từ độc nhất

    # Lặp qua từng danh sách các từ trong các tin nhắn
    for tokens in messages:
        # Lặp qua từng từ trong danh sách các từ của tin nhắn hiện tại
        for token in tokens:
            # Kiểm tra xem từ có nằm trong từ điển chưa
            if token not in dictionary:
                # Nếu từ chưa có trong từ điển, thêm từ vào từ điển
                dictionary.append(token)

    return dictionary  # Trả về từ điển chứa tất cả các từ độc nhất

# Chuyển đổi danh sách các từ thành vector đặc trưng dựa trên từ điển
def create_features(tokens, dictionary):
    # Khởi tạo một vector đặc trưng toàn số 0 với chiều dài bằng số lượng từ trong từ điển
    features = np.zeros(len(dictionary))

    # Lặp qua từng từ trong danh sách các từ của tin nhắn
    for token in tokens:
        # Kiểm tra xem từ có nằm trong từ điển không
        if token in dictionary:
            # Nếu từ có trong từ điển, tìm vị trí của từ trong từ điển và tăng giá trị tại vị trí đó trong vector
            features[dictionary.index(token)] += 1

    return features  # Trả về vector đặc trưng với số lần xuất hiện của từng từ trong từ điển


In [ ]:
dictionary = create_dictionary(messages)
X = np.array([create_features(tokens, dictionary) for tokens in messages])


### **Tiền xử lý dữ liệu nhãn**

In [ ]:
le = LabelEncoder()
y = le.fit_transform(labels)
print(f'Classes: {le.classes_}')
print(f'Encoded labels: {y}')

Classes: ['ham' 'spam']
Encoded labels: [0 0 1 ... 0 0 0]


## **5. Chia bộ dữ liệu train/val/test**


In [ ]:
VAL_SIZE = 0.2
TEST_SIZE = 0.125
SEED = 0

X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=VAL_SIZE,
                                                  shuffle=True,
                                                  random_state=SEED)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
                                                    test_size=TEST_SIZE,
                                                    shuffle=True,
                                                    random_state=SEED)

In [ ]:
print(f'Number of training examples: {X_train.shape[0]}')
print(f'Number of validation examples: {X_val.shape[0]}')
print(f'Number of testing examples: {X_test.shape[0]}')

Number of training examples: 3899
Number of validation examples: 1115
Number of testing examples: 558


## **6. Huấn luyện mô hình**

In [ ]:
%%time
model = GaussianNB()
print('Start training...')
model = model.fit(X_train, y_train)
print('Training completed!')

Start training...
Training completed!
CPU times: user 355 ms, sys: 230 ms, total: 586 ms
Wall time: 577 ms


## **7. Đánh giá mô hình**

In [ ]:
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Val accuracy: {val_accuracy}')
print(f'Test accuracy: {test_accuracy}')

Val accuracy: 0.8816143497757848
Test accuracy: 0.8602150537634409


## **8. Thực hiện dự đoán**

In [ ]:
def predict(text, model, dictionary):
    processed_text = preprocess_text(text)
    features = create_features(text, dictionary)
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    prediction_cls = le.inverse_transform(prediction)[0]

    return prediction_cls

In [ ]:
test_input = 'Hong Kiet is a handsome'
prediction_cls = predict(test_input, model, dictionary)
print(f'Prediction: {prediction_cls}')

Prediction: ham
